# Q1 Data Preprocessing & Data Statistics

Download data from Stock Trading, Financial Statements and Financial Indicators databases on CSMAR. There are 3 raw dataset to be processed:

- **monthly_stock.csv**: monthly stock prices, stock returns, market value of tradable shares from Jan. 2000 to Sep. 2023

- **quarterly_asset.csv**: quarterly total assets, total liabilities, earnings per share, ROA (return on asset), ROE (return on equity), R&D expenses from 2000Q1 to 2023Q3

- **est_mar.csv**: establishment date and market type

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import ast


In [2]:
monthly_stock = pd.read_csv('/Users/Zhuanz/Downloads/A1_FIN3080_121090037/monthly_stock.csv').dropna(axis = 0, how = 'any')
quarterly_asset = pd.read_csv('/Users/Zhuanz/Downloads/A1_FIN3080_121090037/quarterly_asset.csv').dropna(axis = 0, how = 'any')
quarterly_asset = quarterly_asset.query("`Statement Type` != 'B'")  # Only A is selected
est_mar = pd.read_csv('/Users/Zhuanz/Downloads/A1_FIN3080_121090037/est_mar.csv').dropna(axis = 0, how = 'any')

Sampling to check the dataset structure.

In [3]:
monthly_stock.dtypes

Stock Code                                        int64
Trading Month                                    object
Monthly Closing Price                           float64
Market Value of Tradable Shares                 float64
Monthly Return With Cash Dividend Reinvested    float64
dtype: object

In [4]:
quarterly_asset.dtypes

Stock Code                     int64
Ending Date of Statistics     object
Statement Type                object
Total Assets                 float64
Total Liabilities            float64
Stock Short Name              object
R&D Expenses                 float64
Stock Short Name.1            object
Earnings per Share - TTM1    float64
Net Assets per Share         float64
Stock Short Name.2            object
Return on Assets - B         float64
Return on Equity - B         float64
dtype: object

In [5]:
est_mar.dtypes

Stock Code                 int64
Stock Acronym             object
Date Listed (Of Stock)    object
Establishment Date        object
Market Type                int64
dtype: object

Specify quarter column in each dataset.

In [6]:

monthly_stock["Trading Month"] = pd.to_datetime(monthly_stock["Trading Month"], format='%b-%y') # convert strings into datetime
monthly_stock["Quarter"] = monthly_stock["Trading Month"].dt.to_period('Q') - 1 

quarterly_asset["Ending Date of Statistics"] = pd.to_datetime(quarterly_asset["Ending Date of Statistics"])
quarterly_asset["Quarter"] = quarterly_asset["Ending Date of Statistics"].dt.to_period('Q')

est_mar["Date Listed (Of Stock)"] = pd.to_datetime(est_mar["Date Listed (Of Stock)"])
est_mar["Establishment Date"] = pd.to_datetime(est_mar["Establishment Date"])
est_mar["Quarter"]= est_mar["Establishment Date"].dt.to_period('Q')


Merge data into one dataset based on monthly data.

In [7]:
df1 = pd.merge(quarterly_asset, monthly_stock, how="left", left_on=["Stock Code", "Quarter"], right_on=["Stock Code", "Quarter"])
df1.dtypes

Stock Code                                               int64
Ending Date of Statistics                       datetime64[ns]
Statement Type                                          object
Total Assets                                           float64
Total Liabilities                                      float64
Stock Short Name                                        object
R&D Expenses                                           float64
Stock Short Name.1                                      object
Earnings per Share - TTM1                              float64
Net Assets per Share                                   float64
Stock Short Name.2                                      object
Return on Assets - B                                   float64
Return on Equity - B                                   float64
Quarter                                          period[Q-DEC]
Trading Month                                   datetime64[ns]
Monthly Closing Price                                  

## (a) 

#### P/E Ratios

In [8]:
df1["P/E Ratios"] = df1["Monthly Closing Price"]/(df1["Earnings per Share - TTM1"]/3)
df1.sample(3)

,Stock Code,Ending Date of Statistics,Statement Type,Total Assets,Total Liabilities,Stock Short Name,R&D Expenses,Stock Short Name.1,Earnings per Share - TTM1,Net Assets per Share,Stock Short Name.2,Return on Assets - B,Return on Equity - B,Quarter,Trading Month,Monthly Closing Price,Market Value of Tradable Shares,Monthly Return With Cash Dividend Reinvested,P/E Ratios
74770,3006,2023-06-30,A,1.683458e+09,4.050362e+08,BYGF,26105719.00,BYGF,0.560422,2.977219,BYGF,0.077316,0.103183,2023Q2,2023-09-01,14.75,6311935.05,0.061915,78.958356
154469,600597,2022-12-31,A,2.445234e+10,1.390408e+10,GMRY,84661532.00,GMRY,0.283726,7.651197,GMRY,0.016331,0.037431,2022Q4,2023-03-01,10.69,13883229.49,-0.026412,113.031587
131104,430047,2022-09-30,A,3.354978e+08,8.565177e+07,NSLD,42768102.61,NSLD,-0.242550,0.969454,NSLD,-0.148097,-0.191833,2022Q3,2022-10-01,10.69,1595583.50,-0.009268,-132.220161


#### P/B Ratios

In [9]:
df1["P/B Ratios"] = df1["Monthly Closing Price"]/df1["Net Assets per Share"]
df1.sample(3)

,Stock Code,Ending Date of Statistics,Statement Type,Total Assets,Total Liabilities,Stock Short Name,R&D Expenses,Stock Short Name.1,Earnings per Share - TTM1,Net Assets per Share,Stock Short Name.2,Return on Assets - B,Return on Equity - B,Quarter,Trading Month,Monthly Closing Price,Market Value of Tradable Shares,Monthly Return With Cash Dividend Reinvested,P/E Ratios,P/B Ratios
46999,2458,2022-06-30,A,5.564507e+09,2.709855e+09,YSGF,4.005766e+07,YSGF,-0.651554,2.875035,YSGF,-0.068195,-0.125770,2022Q2,2022-07-01,9.29,6273037.04,-0.008538,-42.774659,3.231265
208971,688022,2020-09-30,A,1.418553e+09,5.495397e+08,HCZN,3.345796e+07,HCZN,0.504985,8.046424,HCZN,0.016178,0.023761,2020Q3,2020-11-01,34.55,2011722.78,-0.015670,205.253621,4.293833
26103,2087,2019-09-30,A,9.407658e+09,5.281723e+09,XYFZ,1.055222e+08,XYFZ,0.326970,5.051375,XYFZ,0.020407,0.048250,2019Q3,2019-12-01,3.57,2911829.95,0.043860,32.755299,0.706738


#### Quarterly R&D expense/total asset ratios

In [10]:
quarterly_asset["Quarterly R&D expense/total asset ratios"] = quarterly_asset["R&D Expenses"]/quarterly_asset["Total Assets"]
quarterly_asset.sample(3)

,Stock Code,Ending Date of Statistics,Statement Type,Total Assets,Total Liabilities,Stock Short Name,R&D Expenses,Stock Short Name.1,Earnings per Share - TTM1,Net Assets per Share,Stock Short Name.2,Return on Assets - B,Return on Equity - B,Quarter,Quarterly R&D expense/total asset ratios
25527,526,2022-03-31,A,3.456852e+09,3.040566e+09,XDJY,6726397.24,XDJY,-4.603904,3.534954,XDJY,0.000380,0.003161,2022Q1,0.001946
83130,911,2021-06-30,A,5.472790e+09,5.214234e+09,NNTY,1715167.70,NNTY,0.034112,0.797812,NNTY,0.000749,0.016509,2021Q2,0.000313
269302,300514,2020-06-30,A,9.347601e+08,3.572893e+08,YXD,23816448.76,YXD,0.295958,2.887354,YXD,0.014944,0.027065,2020Q2,0.025479


#### Quarterly Firm Age

In [11]:
current_quarter = pd.to_datetime('now').to_period('Q')
est_mar["Quarterly Firm Age"] = (current_quarter - est_mar["Quarter"]).apply(lambda x: x.n)
est_mar.sample(3)

,Stock Code,Stock Acronym,Date Listed (Of Stock),Establishment Date,Market Type,Quarter,Quarterly Firm Age
690,2556,HLGF,2011-03-02,2004-04-08,4,2004Q2,83
2056,300916,LTZN,2020-12-02,2003-08-29,16,2003Q3,86
2000,300858,KTSW,2020-07-27,2003-09-05,16,2003Q3,86


In [12]:
df1.to_csv('/Users/Zhuanz/Downloads/A1_FIN3080_121090037/new_i.csv', index=False)
quarterly_asset.to_csv('/Users/Zhuanz/Downloads/A1_FIN3080_121090037/new_ii.csv', index=False)
est_mar.to_csv('/Users/Zhuanz/Downloads/A1_FIN3080_121090037/new_iii.csv', index=False)

## (b)

To create statistics summary by market type. Market type corresponds to main board and GEM board. GEM refers to Global Emerging Markets Board. 

In [13]:
def markettype(x):
    if x in [1, 2, 4, 8, 64]:
        return 'main board'
    elif x in [16, 32]:
        return 'GEM board'
    else:
        return x

est_mar['Board Category'] = est_mar['Market Type'].apply(lambda x: markettype(x))
group_est_mar = est_mar.groupby('Board Category') # Group By Board name to calculate

# Create mapping dictionary to map stock code and market type
map_dict = est_mar.set_index('Stock Code')['Board Category'].to_dict()

# Create column 'Board Category' for df1 and quarterly
df1['Board Category'] = df1['Stock Code'].map(map_dict)
quarterly_asset['Board Category'] = quarterly_asset['Stock Code'].map(map_dict)

group_df1 = df1.groupby('Board Category')
group_quarterly_asset = quarterly_asset.groupby('Board Category')


### Summary Statistics Output

##### Monthly Stock Returns

In [14]:
stock_return_describe = group_df1['Monthly Return With Cash Dividend Reinvested'].describe().transpose()
stock_return_describe['Attribute'] = 'Monthly Return'
stock_return_describe

Board Category,GEM board,main board,Attribute
count,66463.000000,118448.000000,Monthly Return
mean,0.008670,0.010579,Monthly Return
std,0.148991,0.133392,Monthly Return
min,-0.836559,-0.882166,Monthly Return
25%,-0.081927,-0.066865,Monthly Return
50%,-0.004777,-0.003168,Monthly Return
75%,0.076660,0.068039,Monthly Return
max,4.256410,2.985951,Monthly Return


##### Monthly P/E Ratio

In [15]:
pe_describe = group_df1["P/E Ratios"].describe().transpose()
pe_describe['Attribute'] = 'P/E Ratios'
pe_describe

Board Category,GEM board,main board,Attribute
count,6.646300e+04,1.184480e+05,P/E Ratios
mean,3.216619e+02,1.689981e+02,P/E Ratios
std,4.050073e+04,8.233213e+03,P/E Ratios
min,-2.078571e+06,-2.063514e+05,P/E Ratios
25%,5.844357e+01,3.547866e+01,P/E Ratios
50%,1.101176e+02,7.082845e+01,P/E Ratios
75%,1.955246e+02,1.309400e+02,P/E Ratios
max,6.262500e+06,1.165000e+06,P/E Ratios


##### Monthly P/B Ratio

In [16]:
pb_describe = group_df1["P/B Ratios"].describe().transpose()
pb_describe['Attribute'] = 'P/B Ratios'
pb_describe

Board Category,GEM board,main board,Attribute
count,66463.000000,118448.000000,P/B Ratios
mean,4.355562,3.069563,P/B Ratios
std,7.565923,20.965678,P/B Ratios
min,-733.511140,-3274.956217,P/B Ratios
25%,2.243390,1.469963,P/B Ratios
50%,3.203396,2.200633,P/B Ratios
75%,4.932938,3.406164,P/B Ratios
max,345.769535,1470.373080,P/B Ratios


##### Quarterly ROA

In [17]:
roa_describe = group_quarterly_asset['Return on Assets - B'].describe().transpose()
roa_describe['Attribute'] = 'ROA'
roa_describe

Board Category,GEM board,main board,Attribute
count,24023.000000,43551.000000,ROA
mean,0.027218,0.030290,ROA
std,0.070056,0.058578,ROA
min,-1.870100,-1.129612,ROA
25%,0.004488,0.006420,ROA
50%,0.021273,0.022438,ROA
75%,0.050405,0.049718,ROA
max,0.968640,1.332036,ROA


##### Quarterly ROE

In [18]:
roe_describe = group_quarterly_asset['Return on Equity - B'].describe().transpose()
roe_describe['Attribute'] = 'ROE'
roe_describe

Board Category,GEM board,main board,Attribute
count,24023.000000,43551.000000,ROE
mean,0.029354,0.039589,ROE
std,0.249411,0.479376,ROE
min,-21.998007,-65.636688,ROE
25%,0.007165,0.012418,ROE
50%,0.032776,0.040507,ROE
75%,0.075233,0.085460,ROE
max,1.319279,6.333252,ROE


##### Quarterly RD Ratio

In [19]:
rd_describe = group_quarterly_asset['Quarterly R&D expense/total asset ratios'].describe().transpose()
rd_describe['Attribute'] = 'RD Ratio'
rd_describe

Board Category,GEM board,main board,Attribute
count,24023.000000,43551.000000,RD Ratio
mean,0.021482,0.014408,RD Ratio
std,0.029234,0.016877,RD Ratio
min,0.000000,-0.002272,RD Ratio
25%,0.007613,0.003651,RD Ratio
50%,0.015209,0.009706,RD Ratio
75%,0.027068,0.019627,RD Ratio
max,2.695974,0.272267,RD Ratio


##### Firm Age

In [20]:
firm_ages_describe = group_est_mar['Quarterly Firm Age'].describe().transpose()
firm_ages_describe['Attribute'] = 'Firm Ages'
firm_ages_describe

Board Category,GEM board,main board,Attribute
count,1910.000000,2699.000000,Firm Ages
mean,82.734031,95.566506,Firm Ages
std,22.705333,22.449175,Firm Ages
min,28.000000,29.000000,Firm Ages
25%,67.000000,81.000000,Firm Ages
50%,83.000000,96.000000,Firm Ages
75%,96.000000,108.000000,Firm Ages
max,176.000000,268.000000,Firm Ages
